# Model Comparison: CheXNet vs. Improved Pipeline

This notebook loads the trained weights from both the Baseline CheXNet and the Improved Pipeline and compares them using various metrics:
1.  **ROC Curves**
2.  **AUC Scores (Per-class and Mean)**
3.  **Inference Speed**
4.  **Precision-Recall Curves**
5.  **Confusion Matrices**
6.  **Calibration Plots**
7.  **Worst Predictions Analysis**

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score, precision_recall_curve, average_precision_score, confusion_matrix
from sklearn.calibration import calibration_curve

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

plt.style.use('ggplot')
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 1. Setup Data and Models

In [ ]:
# --- Data Setup ---
BASE_DIR = '.'
CSV_PATH = os.path.join(BASE_DIR, 'Data_Entry_2017_v2020.csv')
IMAGE_DIRS = [os.path.join(BASE_DIR, 'images'), os.path.join(BASE_DIR, 'images 2')]

df = pd.read_csv(CSV_PATH)
image_paths = {}
for img_dir in IMAGE_DIRS:
    if os.path.exists(img_dir):
        for fname in os.listdir(img_dir):
            if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths[fname] = os.path.join(img_dir, fname)

df['path'] = df['Image Index'].map(image_paths)
df = df.dropna(subset=['path'])

all_labels = ['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia',
              'Pneumothorax', 'Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia']

for label in all_labels:
    df[label] = df['Finding Labels'].map(lambda x: 1.0 if label in x else 0.0)

_, val_df = train_test_split(df, test_size=0.2, random_state=42)
print(f"Validation set size: {len(val_df)}")

class ChestXrayDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.labels = all_labels

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        img_path = row['path']
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = torch.tensor([row[l] for l in self.labels], dtype=torch.float32)
        return image, label

In [ ]:
# --- Model Definitions ---

# 1. CheXNet Baseline
class CheXNet(nn.Module):
    def __init__(self, num_classes):
        super(CheXNet, self).__init__()
        self.densenet = models.densenet121(weights=None) 
        num_ftrs = self.densenet.classifier.in_features
        self.densenet.classifier = nn.Sequential(
            nn.Linear(num_ftrs, num_classes),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.densenet(x)

# 2. Improved Model
class ImprovedModel(nn.Module):
    def __init__(self, num_classes):
        super(ImprovedModel, self).__init__()
        self.backbone = models.efficientnet_v2_s(weights=None)
        num_ftrs = self.backbone.classifier[1].in_features
        self.backbone.classifier[1] = nn.Sequential(
            nn.Linear(num_ftrs, num_classes),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.backbone(x)

In [ ]:
# --- Load Weights ---
baseline_model = CheXNet(len(all_labels)).to(device)
improved_model = ImprovedModel(len(all_labels)).to(device)

try:
    baseline_model.load_state_dict(torch.load('chexnet_baseline.pth', map_location=device))
    print("Loaded CheXNet Baseline weights.")
except FileNotFoundError:
    print("WARNING: 'chexnet_baseline.pth' not found. Model initialized with random weights.")

try:
    improved_model.load_state_dict(torch.load('improved_model.pth', map_location=device))
    print("Loaded Improved Model weights.")
except FileNotFoundError:
    print("WARNING: 'improved_model.pth' not found. Model initialized with random weights.")

baseline_model.eval()
improved_model.eval();

## 2. Generate Predictions

In [ ]:
# Transforms
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# Note: Models expect different input sizes
transform_baseline = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(), normalize])
transform_improved = transforms.Compose([transforms.Resize((320, 320)), transforms.ToTensor(), normalize])

ds_baseline = ChestXrayDataset(val_df, transform=transform_baseline)
ds_improved = ChestXrayDataset(val_df, transform=transform_improved)

dl_baseline = DataLoader(ds_baseline, batch_size=32, shuffle=False, num_workers=0)
dl_improved = DataLoader(ds_improved, batch_size=32, shuffle=False, num_workers=0)

def get_predictions(model, loader):
    all_preds = []
    all_targets = []
    start_time = time.time()
    
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            all_preds.append(outputs.cpu().numpy())
            all_targets.append(labels.numpy())
            
    end_time = time.time()
    return np.concatenate(all_preds), np.concatenate(all_targets), end_time - start_time

print("Running inference for Baseline...")
preds_base, targets_base, time_base = get_predictions(baseline_model, dl_baseline)

print("Running inference for Improved Model...")
preds_imp, targets_imp, time_imp = get_predictions(improved_model, dl_improved)

print(f"Baseline Inference Time: {time_base:.2f}s")
print(f"Improved Inference Time: {time_imp:.2f}s")

## 3. Comparative Metrics

In [ ]:
# Calculate AUCs
auc_base = []
auc_imp = []

for i in range(len(all_labels)):
    try:
        auc_base.append(roc_auc_score(targets_base[:, i], preds_base[:, i]))
    except: auc_base.append(0.5)
        
    try:
        auc_imp.append(roc_auc_score(targets_imp[:, i], preds_imp[:, i]))
    except: auc_imp.append(0.5)

# Create Comparison DataFrame
comp_df = pd.DataFrame({
    'Label': all_labels,
    'Baseline AUC': auc_base,
    'Improved AUC': auc_imp
})

comp_df['Difference'] = comp_df['Improved AUC'] - comp_df['Baseline AUC']
print(f"Mean Baseline AUC: {np.mean(auc_base):.4f}")
print(f"Mean Improved AUC: {np.mean(auc_imp):.4f}")
comp_df.sort_values(by='Difference', ascending=False)

In [ ]:
# Plot AUC Comparison
plt.figure(figsize=(14, 6))
x = np.arange(len(all_labels))
width = 0.35

plt.bar(x - width/2, auc_base, width, label='Baseline (DenseNet)', color='skyblue')
plt.bar(x + width/2, auc_imp, width, label='Improved (EfficientNetV2)', color='salmon')

plt.xlabel('Conditions')
plt.ylabel('AUC Score')
plt.title('Per-Class AUC Comparison')
plt.xticks(x, all_labels, rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.show()

## 4. ROC Curves

In [ ]:
# Plot ROC for top 4 classes (by improvement)
top_indices = comp_df.sort_values(by='Difference', ascending=False).head(4).index

plt.figure(figsize=(15, 12))
for i, idx in enumerate(top_indices):
    label = all_labels[idx]
    fpr_b, tpr_b, _ = roc_curve(targets_base[:, idx], preds_base[:, idx])
    fpr_i, tpr_i, _ = roc_curve(targets_imp[:, idx], preds_imp[:, idx])
    
    plt.subplot(2, 2, i+1)
    plt.plot(fpr_b, tpr_b, label=f'Baseline (AUC={auc_base[idx]:.2f})')
    plt.plot(fpr_i, tpr_i, label=f'Improved (AUC={auc_imp[idx]:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve: {label}')
    plt.legend()

plt.tight_layout()
plt.show()

## 5. Precision-Recall Curves
Crucial for imbalanced datasets.

In [ ]:
plt.figure(figsize=(15, 12))
for i, idx in enumerate(top_indices):
    label = all_labels[idx]
    precision_b, recall_b, _ = precision_recall_curve(targets_base[:, idx], preds_base[:, idx])
    precision_i, recall_i, _ = precision_recall_curve(targets_imp[:, idx], preds_imp[:, idx])
    
    ap_b = average_precision_score(targets_base[:, idx], preds_base[:, idx])
    ap_i = average_precision_score(targets_imp[:, idx], preds_imp[:, idx])
    
    plt.subplot(2, 2, i+1)
    plt.plot(recall_b, precision_b, label=f'Baseline (AP={ap_b:.2f})')
    plt.plot(recall_i, precision_i, label=f'Improved (AP={ap_i:.2f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve: {label}')
    plt.legend()

plt.tight_layout()
plt.show()

## 6. Confusion Matrices (Threshold = 0.5)
Visualizing True Positives, False Positives, etc.

In [ ]:
threshold = 0.5
idx = top_indices[0] # Look at the most improved class
label = all_labels[idx]

cm_base = confusion_matrix(targets_base[:, idx], (preds_base[:, idx] > threshold).astype(int))
cm_imp = confusion_matrix(targets_imp[:, idx], (preds_imp[:, idx] > threshold).astype(int))

fig, ax = plt.subplots(1, 2, figsize=(12, 5))
sns.heatmap(cm_base, annot=True, fmt='d', cmap='Blues', ax=ax[0])
ax[0].set_title(f'Baseline CM ({label})')
ax[0].set_ylabel('True Label')
ax[0].set_xlabel('Predicted Label')

sns.heatmap(cm_imp, annot=True, fmt='d', cmap='Greens', ax=ax[1])
ax[1].set_title(f'Improved CM ({label})')
ax[1].set_ylabel('True Label')
ax[1].set_xlabel('Predicted Label')

plt.show()

## 7. Calibration Plots
How well do the predicted probabilities match the actual frequency of positives?

In [ ]:
plt.figure(figsize=(10, 8))
idx = top_indices[0]
label = all_labels[idx]

prob_true_b, prob_pred_b = calibration_curve(targets_base[:, idx], preds_base[:, idx], n_bins=10)
prob_true_i, prob_pred_i = calibration_curve(targets_imp[:, idx], preds_imp[:, idx], n_bins=10)

plt.plot(prob_pred_b, prob_true_b, marker='o', label='Baseline')
plt.plot(prob_pred_i, prob_true_i, marker='o', label='Improved')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('Mean Predicted Probability')
plt.ylabel('Fraction of Positives')
plt.title(f'Calibration Curve: {label}')
plt.legend()
plt.show()